In [1]:
import logging

import numpy

import cicada.additive
import cicada.communicator

logging.basicConfig(level=logging.INFO)

@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)

    # Player 0 will provide the secret divisor.
    a = numpy.array(144409) if communicator.rank == 0 else None
    # Every player knows the modulus.
    b = numpy.array(117)

    log.info(f"Dividend: {a}", src=0)
    log.info(f"Modulus: {b}", src=1)

    a_share = protocol.share(src=0, secret=protocol.encoder.encode(a), shape=())
    remainder_share = protocol.modulus_private_public(a_share, b)
    remainder = protocol.encoder.decode(protocol.reveal(remainder_share))

    log.info(f"Player {communicator.rank} remainder: {remainder}")

main()

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:64610 from tcp://127.0.0.1:64610.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:64610 from tcp://127.0.0.1:64611.
INFO:cicada.communicator.nng:Player 2 rendezvous with tcp://127.0.0.1:64610 from tcp://127.0.0.1:64612.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator ready.
INFO:root:Dividend: 144409
INFO:root:Modulus: 117
INFO:root:Player 0 remainder: 31.0
INFO:root:Player 1 remainder: 31.0
INFO:root:Player 2 remainder: 31.0
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator freed.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator freed.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator freed.
INFO:cicada.communicator.nng:Player 0 returned: None
INFO:cicada.communicator.nng:Player 1 returned: None
I

[None, None, None]

As you can see, the result revealed to the players is correct.